In [1]:
import sys
import time
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel.model.module import rnn_cells
from seqmodel.model import rnn_module

In [25]:
tf.reset_default_graph()
batch_size = 4
max_time = 5
dim = 3
cell_dim = 2
input_data = np.random.randn(max_time, batch_size, dim).astype(np.float32)
seq_len_data = np.array([4, 3, 2, 1], dtype=np.int32)
first_output = np.random.randn(batch_size, cell_dim).astype(np.float32)
inputs = tf.constant(input_data, dtype=tf.float32)
seq_len = tf.constant(seq_len_data)
share_rnn_opt = rnn_module.BasicRNNModule.default_opt()
share_rnn_opt.rnn_cell.cell_opt.num_units = cell_dim
share_rnn_opt.create_zero_initial_state = True
share_rnn_opt.logit=None
share_rnn_opt.rnn_cell.cell_class
enc = rnn_module.BasicRNNModule(share_rnn_opt, name='encoder')(inputs, seq_len)
for v in tf.trainable_variables():
    print(v.name)

encoder/rnn/basic_lstm_cell/weights:0
encoder/rnn/basic_lstm_cell/biases:0


In [65]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())


In [71]:
zero_state = sess.run(enc.initial_state)
res = sess.run([enc.initial_state, enc.final_state, enc.cell_output], {seq_len:seq_len_data, 
                     enc.initial_state.output:first_output, 
                     enc.initial_state.state:zero_state.state})
print(res)

[OutputToInputStateTuple(state=LSTMStateTuple(c=array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32), h=array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32)), output=array([[ 0.0785614 ,  0.51012033],
       [ 0.56420118, -0.75726998],
       [ 0.25703451,  1.73136818],
       [-0.17974873,  0.48394176]], dtype=float32)), OutputToInputStateTuple(state=LSTMStateTuple(c=array([[ 0.05315012, -0.03422486],
       [ 0.06633939,  0.06921557],
       [ 0.24002026, -0.20174295],
       [-0.03293049, -0.0953301 ]], dtype=float32), h=array([[ 0.02638773, -0.01719238],
       [ 0.03324401,  0.03618468],
       [ 0.11379175, -0.09651362],
       [-0.01517337, -0.04817434]], dtype=float32)), output=array([[ 0.02638773, -0.01719238],
       [ 0.03324401,  0.03618468],
       [ 0.11379175, -0.09651362],
       [-0.01517337, -0.04817434]], dtype=float32)), array([[[ 0.02904397, -0.06188283],
        [ 0.07783409,  0.06721009

In [11]:
print(enc.to_pretty())

cell_output: Tensor("encoder_1/rnn/TensorArrayStack/TensorArrayGatherV3:0", shape=(5, 4, 2), dtype=float32)
final_state: OutputToInputStateTuple(state=LSTMStateTuple(c=<tf.Tensor 'encoder_1/rnn/while/Exit_2:0' shape=(?, 2) dtype=float32>, h=<tf.Tensor 'encoder_1/rnn/while/Exit_3:0' shape=(?, 2) dtype=float32>), output=<tf.Tensor 'encoder_1/rnn/while/Exit_4:0' shape=(?, 2) dtype=float32>)
initial_state: OutputToInputStateTuple(state=LSTMStateTuple(c=<tf.Tensor 'encoder_1/zeros:0' shape=(?, 2) dtype=float32>, h=<tf.Tensor 'encoder_1/zeros_1:0' shape=(?, 2) dtype=float32>), output=<tf.Tensor 'encoder_1/zeros_2:0' shape=(?, 2) dtype=float32>)


In [34]:
zero_state

OutputToInputStateTuple(state=LSTMStateTuple(c=array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32), h=array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32)), output=array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32))